In [8]:
import pandas as pd
import numpy as np
import math
import plotly
import plotly.graph_objects as go

In [2]:
class master_track:
    def __init__(self, track_paths, play_info_path, play_details_path, players_path):
        self.d_line_pos = ["NT", "DT", "MLB", "ILB", "LB",  "OLB"]
        self.o_line_pos = ["T", "C", "G"]
        self.qb_pos = ["QB"]

        player_df = pd.read_csv(players_path)
        self.player_df = player_df
        self.o_linemen_player_ids = self.player_df.loc[self.player_df['officialPosition'].isin(self.o_line_pos)].nflId.unique()
        self.d_linemen_player_ids = self.player_df.loc[self.player_df['officialPosition'].isin(self.d_line_pos)].nflId.unique()
        self.all_linemen = np.append(self.o_linemen_player_ids, self.d_linemen_player_ids)

        self.play_info_df = pd.read_csv(play_info_path)
        self.play_info_df['playId'] = (self.play_info_df['gameId'].apply(str) + self.play_info_df['playId'].apply(str)).apply(int)
        self.play_details_df = pd.read_csv(play_details_path)
        self.play_details_df['playId'] = (self.play_details_df['gameId'].apply(str) + self.play_details_df['playId'].apply(str)).apply(int)
        self.play_details_df['disrupt_individual'] = (self.play_details_df.pff_hit + self.play_details_df.pff_hurry + self.play_details_df.pff_sack) > 0
        team_dets = self.play_details_df.groupby('playId').agg({'disrupt_individual' : [sum]})
        team_dets.columns = ["_".join(x) for x in np.array(team_dets.columns).ravel()]
        team_dets = team_dets.reset_index()
        team_dets['disrupt_team'] = team_dets.disrupt_individual_sum > 0
        team_dets = team_dets.loc[:,['playId', 'disrupt_team']].drop_duplicates()
        self.play_details_df = self.play_details_df.merge(team_dets, on = 'playId', how = 'left').copy()
        self.max_time_after_snap = 7
        
        self.track_dfs = {}
        week_num = 1
        for week_path in track_paths:
            week = pd.read_csv(week_path)
            week['playId'] = (week['gameId'].apply(str) + week['playId'].apply(str)).apply(int)
            week['week'] = week_num
            snap_frames_df = week.loc[week.event == 'ball_snap', ['playId', 'frameId']].drop_duplicates().reset_index(drop  = True).rename(columns = {'frameId' : 'snap_frame'})
            week = week.merge(snap_frames_df, on = 'playId', how = 'left')
            week['time_after_snap'] = (week.frameId - week.snap_frame) * 0.1
            no_snap = week.loc[week.time_after_snap.isnull()].drop_duplicates().playId.tolist()
            week = week.loc[~(week.playId.isin(no_snap))]
            self.track_dfs.update( {''.join(filter(lambda i: i.isdigit(), week_path)) : week} )
            week_num += 1

        self.individual_play_avgs = {key : None for key in self.track_dfs.keys()}
        self.training_data_individual = {key : None for key in self.track_dfs.keys()}
        self.training_data_team = {key : None for key in self.track_dfs.keys()}


    def search_track_weeks(self, variables, variable_values):
        needed = pd.DataFrame()
        for week in range (len(self.track_dfs)):
            week = str(week + 1)
            curr_week = self.track_dfs.get(week)
            if len(variables) == 2:
                needed_info = curr_week.loc[(curr_week[variables[0]] == variable_values[0]) & (curr_week[variables[1]] == variable_values[1])]
            else:
                needed_info = curr_week.loc[(curr_week[variables[0]] == variable_values[0])]
            needed = pd.concat([needed, needed_info])
        return(needed.reset_index(drop = True))

    def get_qb_track_on_play(self, play_id):
        qb_play = self.play_details_df
        qb_id = qb_play.loc[(qb_play.playId == play_id) & (qb_play.pff_positionLinedUp == 'QB')].reset_index(drop = 0).nflId[0]

        qb_track = self.search_track_weeks(variables = ["nflId", "playId"], variable_values = [qb_id, play_id]).reset_index(drop = True)
        return(qb_track)

    def get_defender_and_qb_info_on_play(self, play_id):
        # get qb and def data
        # get def track
        def_match = self.play_details_df.loc[(self.play_details_df.playId == play_id) & (self.play_details_df.pff_nflIdBlockedPlayer.notna()), ['playId', 'nflId', 'pff_nflIdBlockedPlayer']]
        defender_track = []
        for defender in def_match.nflId:
            this_def = self.search_track_weeks(variables = ["playId", "nflId"], variable_values = [play_id, defender])
            defender_track.append(this_def)
        defender_track = pd.concat(defender_track).merge(def_match, on = ['playId', 'nflId'], how = 'left').loc[:,['nflId', 'playId', 'time_after_snap', 'pff_nflIdBlockedPlayer', 'x', 'y']].rename(columns = {'pff_nflIdBlockedPlayer' : 'nflId', 'nflId' : 'blockerId', 'x' : 'x_block', 'y' : 'y_block'})

        # get qb track
        qb_track = self.get_qb_track_on_play(play_id).loc[:, ['playId', 'time_after_snap', 'x', 'y', 's', 'a']].rename(columns = {'x' : 'x_qb', 'y' : 'y_qb', 's' : 's_qb', 'a' : 'a_qb'})
        defender_qb_track = defender_track.merge(qb_track, on = ['playId', 'time_after_snap'], how = 'left')
        defender_qb_track['blocker_distance_from_qb'] = np.sqrt( (defender_qb_track.x_qb - defender_qb_track.x_block)**2 + (defender_qb_track.y_qb - defender_qb_track.y_block)**2 )

        dist_at_snap = defender_qb_track.loc[defender_qb_track.time_after_snap == 0, ['blockerId', 'blocker_distance_from_qb']].rename(columns = {'blocker_distance_from_qb' : 'distance_from_qb_at_snap'})
        defender_qb_track = defender_qb_track.merge(dist_at_snap, on = 'blockerId', how = 'left')
        defender_qb_track['blocker_distance_toward_qb_gained'] = defender_qb_track['distance_from_qb_at_snap'] - defender_qb_track['blocker_distance_from_qb']
        def_qb_dat = defender_qb_track.drop(['distance_from_qb_at_snap'], axis = 1)

        # get defender track wide

        defender_track_wide = []
        for column in ['blockerId', 'x_block', 'y_block', 'blocker_distance_from_qb', 'blocker_distance_toward_qb_gained']:
            this_col_widen = def_qb_dat.groupby(['playId', 'time_after_snap', 'nflId'])[column].apply(lambda s: pd.Series(s.values, index=[f'{column}%s' % i for i in range(s.shape[0])])).unstack(-1).reset_index()
            defender_track_wide.append(this_col_widen)
        defender_track_wide = reduce(lambda x, y: pd.merge(x, y, on = ['playId', 'time_after_snap', 'nflId']), defender_track_wide).sort_values(['nflId', 'time_after_snap'])
        def_qb_dat = def_qb_dat.drop(['blockerId', 'x_block', 'y_block', 'blocker_distance_from_qb', 'blocker_distance_toward_qb_gained'], axis = 1).merge(defender_track_wide, on = ['playId', 'time_after_snap', 'nflId'], how = 'left')

        return(def_qb_dat)

    def get_play(self, play_id):
        def_qb_dat = self.get_defender_and_qb_info_on_play(play_id)
        # separate qb and def data 
        qb_dat = def_qb_dat.loc[:,['time_after_snap', 'x_qb', 'y_qb', 's_qb', 'a_qb']].drop_duplicates()
        def_dat = def_qb_dat.drop(['x_qb', 'y_qb', 's_qb', 'a_qb'], axis = 1)
        #get pass rusher
        rush_ids = self.play_details_df.loc[(self.play_details_df.pff_role == "Pass Rush") & (self.play_details_df.playId == play_id)].nflId.tolist()
        play = self.search_track_weeks(variables = ["playId"], variable_values = [play_id])
        play = play.loc[(play.nflId.isin(rush_ids))]
        # create rush qb relationship data
        play = play.merge(qb_dat, on = 'time_after_snap', how ='left')
        play['rusher_distance_from_qb'] = np.sqrt( (play.x_qb - play.x)**2 + (play.y_qb - play.y)**2 )
        dist_at_snap = play.loc[play.frameId == play.snap_frame, ['nflId', 'rusher_distance_from_qb']].rename(columns = {'rusher_distance_from_qb' : 'distance_from_qb_at_snap'})
        play = play.merge(dist_at_snap, on = 'nflId', how = 'left')
        play['rusher_distance_toward_qb_gained'] = play.distance_from_qb_at_snap - play.rusher_distance_from_qb
        dis_gained_this_play_to_qb, change_in_velocity_this_play = [np.NaN], [np.NaN]
        dis_gained_this_play_to_qb.extend(np.diff(play.rusher_distance_from_qb))
        play['rusher_velocity_towards_qb'] = np.array(dis_gained_this_play_to_qb) / 0.1
        change_in_velocity_this_play.extend(np.diff(play.rusher_velocity_towards_qb))
        play['rusher_acceleration_towards_qb'] = np.array(change_in_velocity_this_play) / 0.1
        #get rusher blocker data ralations
        play_w_dup_blockers = play.merge(def_dat, on = ['playId', 'time_after_snap', 'nflId'], how = 'left')
        play_w_dup_blockers.loc[play_w_dup_blockers.blockerId0.notna()]
        all_blockers_accounted_for = False
        id = 0
        while all_blockers_accounted_for == False:
            try:
                play_w_dup_blockers['blocker_in_front' + str(id)] = (play_w_dup_blockers['blocker_distance_from_qb' + str(id)] - play_w_dup_blockers['rusher_distance_from_qb']) < 0
                play_w_dup_blockers['blocker_distance_from_rusher' + str(id)] = np.sqrt( (play_w_dup_blockers['x_block' + str(id)] - play_w_dup_blockers['x'])**2 + (play_w_dup_blockers['y_block' + str(id)] - play_w_dup_blockers['y'])**2 )
                id += 1
            except:
                all_blockers_accounted_for = True
        play_w_dup_blockers['blockers_left'] = sum([play_w_dup_blockers['blocker_in_front' + str(this_id)] for this_id in range(id)])
        play_w_dup_blockers['number_blockers_on_play'] = sum([play_w_dup_blockers['blockerId' + str(this_id)].notna() for this_id in range(id)])
        [play_w_dup_blockers['blocker_distance_from_rusher' + str(this_id)] for this_id in range(id)]
        play_w_dup_blockers['distance_of_closest_blocker_in_front'] = np.where(play_w_dup_blockers['blockers_left'] > 0, play_w_dup_blockers.loc[:,['blocker_distance_from_rusher' + str(this_id) for this_id in range(id)]].min(axis=1), 0)
        play = play_w_dup_blockers.loc[play_w_dup_blockers.time_after_snap >= 0,[
            'gameId', 'playId', 'nflId', 'time_after_snap', 'team', 'week',
            'x', 'y', 'rusher_velocity_towards_qb', 'rusher_acceleration_towards_qb',
            'x_qb', 'y_qb', 's_qb', 'a_qb',
            'rusher_distance_from_qb', 'rusher_distance_toward_qb_gained',  'distance_of_closest_blocker_in_front', 'blockers_left', 'number_blockers_on_play']]
        return(play)

    def load_training_data(self, week):
        if week < 2:
            print("Must check for week after week 1 to have data")
            return(None)
        needed_weeks = list(range(1, week))
        for each_week in needed_weeks:
            print(f'Getting week {each_week} training data')
            if (str(each_week) in self.track_dfs.keys()) and not (self.training_data_individual.get(str(each_week)) is None) :
                next
            else:
                all_individual_df = []
                all_team_df = []
                plays_this_week = np.unique(self.track_dfs.get(str(each_week)).playId).tolist()
                for play_id in tqdm.tqdm(plays_this_week):
                    play_individual_df = self.get_play(play_id = play_id)
                    all_individual_df.append(play_individual_df)
                all_individual_df = pd.concat(all_individual_df)
                all_team_df = all_individual_df.groupby(['playId', 'time_after_snap'], as_index=False).agg({
                    'nflId' : pd.Series.nunique,
                    's_qb' : [min],
                    'a_qb' : [min],
                    'blockers_left' : [min, max, sum],
                    'rusher_velocity_towards_qb' : [min, max, np.average],
                    'rusher_acceleration_towards_qb' : [min, max, np.average],
                    "rusher_distance_from_qb": [min, max, np.average],
                    "rusher_distance_toward_qb_gained": [min, max, np.average],
                    "distance_of_closest_blocker_in_front": [min, max, np.average]})
                all_team_df.columns = ["_".join(x) for x in np.array(all_team_df.columns).ravel()]
                all_team_df = all_team_df.rename(columns = {'playId_' : 'playId', 'time_after_snap_' : 'time_after_snap'}).rename(columns = {'nflId_nunique' : 'n_rushers', 's_qb_min' : 's_qb', 'a_qb_min' : 'a_qb'})
                all_team_df['week'] = each_week

                results_data = self.play_details_df.fillna(0)
                results_data_ind = results_data.loc[:, ['nflId', 'playId', 'pff_positionLinedUp', 'disrupt_individual']]
                results_data_ind.pff_positionLinedUp = np.select(
                            [
                                (['E' in position for position in results_data_ind.pff_positionLinedUp]), 
                                (['B' in position for position in results_data_ind.pff_positionLinedUp]),
                                (['T' in position for position in results_data_ind.pff_positionLinedUp])

                            ], 
                            [
                                'End', 
                                'Back',
                                'Tackle'

                            ], 
                            default='Non-Lineman'
                        )
                results_data_team = results_data.loc[:, ['playId', 'disrupt_team']].drop_duplicates()
                training_df_individual = all_individual_df.merge(results_data_ind, on = ['nflId', 'playId'], how = 'left')
                training_df_team = all_team_df.merge(results_data_team, on = ['playId'], how = 'left')
                
                self.training_data_individual[str(each_week)] = training_df_individual
                self.training_data_team[str(each_week)] = training_df_team
        print('Done')


    def get_averages_up_to_week(self, week):
        if week < 2:
            print("Must check for week after week 1 to have data")
            return(None)
        needed_weeks = list(range(1, week))
        progressive_training_indivudual = []
        for each_week in needed_weeks:
            print(f'Getting week {each_week} average metrics by position and blocker number')
            if (str(each_week) in self.track_dfs.keys()) and not (self.training_data_individual.get(str(each_week)) is None) :
                next
            try:
                this_week_train_dat = self.training_data_individual.get(str(each_week))
            except:
                print(f'Training data has not been loaded for week {str(each_week)}')
                return(None)
            progressive_training_indivudual.append(this_week_train_dat)
            all_before_this_week = pd.concat(progressive_training_indivudual)  
            player_averages = all_before_this_week.groupby(['time_after_snap', 'number_blockers_on_play', 'pff_positionLinedUp']).agg({
                                                            'nflId' : pd.Series.nunique,
                                                            'blockers_left' : [np.median],
                                                            'rusher_velocity_towards_qb' : [np.average],
                                                            'rusher_acceleration_towards_qb' : [np.average],
                                                            "rusher_distance_from_qb": [np.average],
                                                            "rusher_distance_toward_qb_gained": [np.average],
                                                            "distance_of_closest_blocker_in_front": [np.average]
                                                            })
            player_averages.columns = ["_".join(x) for x in np.array(player_averages.columns).ravel()]
            player_averages = player_averages.rename(columns = {'time_after_snap_' : 'time_after_snap'}).rename(columns = {'nflId_nunique' : 'n_rushers'}).reset_index()
            self.individual_play_avgs[str(each_week)] = player_averages
        print('Done')

    def get_rush_sequences_labels(self, week_id = 'all', play_id = 'all', normalize = True, replace_player = None):
        if (play_id == 'all') & (replace_player != None):
            print('Can only replace player for outputs of one play')
            return None
        if ( (week_id != 'all') & (play_id != 'all') ):
            print('Filter on EITHER week or playId but not both.')
            return None
        if (replace_player != None):
            play_week = self.search_track_weeks(variables = ["playId"], variable_values = [play_id]).reset_index(drop = True).week.tolist()[0]
        else:
            play_week = None
        needed_weeks = list(range(1, week))
        progressive_training_team = []
        for each_week in needed_weeks:
            if normalize == True:
                print(f'Getting week {each_week} normalized training data')
            else:
                print(f'Getting week {each_week} (unnormalized) training data')
            try:
                if (replace_player == None) or (each_week != play_week):
                    this_week_train_dat = self.training_data_team.get(str(each_week))
                else:
                    this_week_train_dat = self.training_data_team.get(str(each_week))
                    replaced_player_play = self.replace_player_with_average(play_id, replace_player)
                    this_week_train_dat = this_week_train_dat.loc[this_week_train_dat.playId != play_id]
                    this_week_train_dat = pd.concat([this_week_train_dat, replaced_player_play])
            except:
                print(f'Training data has not been loaded for week {str(each_week)}')
            progressive_training_team.append(this_week_train_dat)

        all_training = pd.concat(progressive_training_team)

        if normalize == True:
                scaler = sklearn.preprocessing.MinMaxScaler()
                scale_cols = all_training.drop(['time_after_snap', 'playId', 'week', 'disrupt_team'], axis = 1)
                non_scale_cols = all_training.loc[:,['time_after_snap', 'playId', 'week', 'disrupt_team']]
                columns = scale_cols.columns
                scale_cols = pd.DataFrame(scaler.fit_transform(scale_cols))
                scale_cols.columns = columns
                all_training = pd.concat([non_scale_cols.reset_index(drop = 1), scale_cols.reset_index(drop = 1)], axis = 1)

        if week_id != 'all':
            all_training = all_training.loc[all_training.week == week_id]
        if play_id != 'all':
            all_training = all_training.loc[all_training.playId == play_id]

        sequences = []
        for playId, group in tqdm.tqdm(all_training.groupby("playId")):
            label = group.iloc[0].disrupt_team
            sequence_features = group.drop(['time_after_snap', 'playId', 'week', 'disrupt_team'], axis = 1)
            sequences.append((sequence_features, label))
        return(sequences)


    def replace_player_with_average(self, play_id, player_id): # might need to add normalize to this

        play_week = all_data.search_track_weeks(variables = ["playId"], variable_values = [play_id]).reset_index(drop = True).week.tolist()[0]
        all_week = all_data.training_data_individual.get(str(play_week))
        play_dat_individual = all_week.loc[(all_week.playId == play_id)]
        lined_up_pos = play_dat_individual.loc[(play_dat_individual.nflId == player_id)].pff_positionLinedUp.tolist()[0]
        num_blockers = play_dat_individual.loc[(play_dat_individual.nflId == player_id)].number_blockers_on_play.tolist()[0]
        max_time_after_snap = max(play_dat_individual.loc[(play_dat_individual.nflId == player_id)].time_after_snap.tolist())
        current_player = play_dat_individual.loc[(play_dat_individual.nflId == player_id)].copy()
        current_player_keep = current_player.loc[:,['gameId', 'playId', 'nflId', 'team', 'week', 'x', 
                                                    'y', 'x_qb', 'y_qb', 's_qb', 'a_qb', 'disrupt_individual']].reset_index(drop = 1)


        all_averages = all_data.individual_play_avgs.get(str(play_week))
        condition_average = all_averages.loc[(all_averages.pff_positionLinedUp == lined_up_pos) & 
                                                (all_averages.number_blockers_on_play == num_blockers) &
                                                (all_averages.time_after_snap <= max_time_after_snap)].reset_index(drop = 1).rename({'blockers_left_median' : 'blockers_left', 
                                                                    'rusher_velocity_towards_qb_average' : 'rusher_velocity_towards_qb',
                                                                    'rusher_acceleration_towards_qb_average' : 'rusher_acceleration_towards_qb', 
                                                                    'rusher_distance_from_qb_average' : 'rusher_distance_from_qb', 
                                                                    'rusher_distance_toward_qb_gained_average' : 'rusher_distance_toward_qb_gained',
                                                                    'distance_of_closest_blocker_in_front_average' : 'distance_of_closest_blocker_in_front'}, axis = 1)

        replace_averages = pd.concat([condition_average, current_player_keep], axis = 1).reset_index(drop = 1)
        removed_player = play_dat_individual.loc[~(play_dat_individual.nflId == player_id)].reset_index(drop = 1)
        replaced = pd.concat([replace_averages, removed_player])
        replaced_team = replaced.groupby(['playId', 'time_after_snap'], as_index=False).agg({
                                    'nflId' : pd.Series.nunique,
                                    's_qb' : [min],
                                    'a_qb' : [min],
                                    'blockers_left' : [min, max, sum],
                                    'rusher_velocity_towards_qb' : [min, max, np.average],
                                    'rusher_acceleration_towards_qb' : [min, max, np.average],
                                    "rusher_distance_from_qb": [min, max, np.average],
                                    "rusher_distance_toward_qb_gained": [min, max, np.average],
                                    "distance_of_closest_blocker_in_front": [min, max, np.average],
                                    "disrupt_individual" : [sum],
                                    "week" : [min]})
        replaced_team.columns = ["_".join(x) for x in np.array(replaced_team.columns).ravel()]
        replaced_team['disrupt_individual_sum'] = replaced_team['disrupt_individual_sum'] > 1
        replaced_team = replaced_team.rename(columns = {'playId_' : 'playId', 'time_after_snap_' : 'time_after_snap'}).rename(columns = {'nflId_nunique' : 'n_rushers', 
                                                                                                                                        's_qb_min' : 's_qb', 'a_qb_min' : 'a_qb',
                                                                                                                                        'disrupt_individual_sum' : 'disrupt_team',
                                                                                                                                        'week_min' : 'week'})
        return(replaced_team)
    



####### modeling objects


## make dataset object

class RushDataset(Dataset):
    def __init__(self, sequences, sequence_length):
        self.sequences = sequences
        self.sequence_length = sequence_length
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        label = int(label)
        labels = np.repeat(label, self.sequence_length)
        if sequence.shape[0] > self.sequence_length:
            sequence = sequence.iloc[0:self.sequence_length]
        if sequence.shape[0] < self.sequence_length:
            # padding = pd.DataFrame(0, index=np.arange(self.sequence_length - sequence.shape[0]), columns=sequence.columns) zero pad does not work
            padding = pd.DataFrame(sequence.iloc[-1:].copy())
            num_padded = self.sequence_length - sequence.shape[0]
            padding_rep = pd.DataFrame(np.repeat(padding.values, num_padded, axis=0)) 
            padding_rep.columns = sequence.columns
            sequence = pd.concat([sequence, padding_rep])

        label_tensor = torch.Tensor(labels).long()
        data_tensor = torch.Tensor(sequence.values)
            
        return data_tensor, label_tensor
    

class SequenceModel(torch.nn.Module):
    def __init__(self, n_features, n_classes, n_hidden, n_layers):
        super().__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.ltsm = torch.nn.LSTM(input_size = n_features, hidden_size = n_hidden, num_layers = n_layers, batch_first = True)
        self.classifier = torch.nn.Linear(n_hidden, n_classes)
    def forward(self, x):
        unfolded_hiddens, (_, _) = self.ltsm(x)
        fc_output = self.classifier(unfolded_hiddens)
        return fc_output
    def get_layer_probabilities(self, x):
        fc_output = self.forward(x)
        fc_output_long = fc_output.view([x.size(0) * x.size(1), self.n_classes])
        time_prob = torch.nn.functional.softmax(fc_output_long, dim = 1).detach().cpu().numpy().tolist()
        time_prob_sack = [x[1] for x in time_prob]
        return time_prob_sack



def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    outputs = []
    labels = []
    time_frame_losses = {}
    time_frame_labels = {}
    time_frame_fc_outputs = {}
    i = 0
    for X, y in data_loader:
        i += 1
        fc_output = model(X)
        fc_output_long = fc_output.view([data_loader.dataset.sequence_length * X.size(0), model.n_classes])
        y_long = y.view([data_loader.dataset.sequence_length * X.size(0)])
        outputs.extend([x[0] for x in fc_output_long.detach().numpy().tolist()])
        labels.extend(y_long.numpy())
        loss = loss_function(fc_output_long, y_long)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # get auc per 5 frames

        time_frame_start = 0
        if data_loader.dataset.sequence_length%5 != 0:
            print('Pick a sequence length divisable by 5')
            return None
        for time_frames in range(int(data_loader.dataset.sequence_length / 5)):
            time_frames += 1
            this_time_frame_fc_output = fc_output[:, time_frame_start : time_frames*5, :].reshape([X.size(0) * 5, 2])
            this_time_frame_ys = y[:, time_frame_start : time_frames*5].reshape([X.size(0) * 5])
            loss_this_time_frame = loss_function(this_time_frame_fc_output, this_time_frame_ys)
            if i == 1:
                time_frame_losses.update({str(time_frames) : [loss_this_time_frame.item()]})
                time_frame_labels.update({str(time_frames) : this_time_frame_ys.detach().cpu().numpy().tolist()})
                time_frame_fc_outputs.update({str(time_frames) : this_time_frame_fc_output.detach().cpu().numpy().tolist()})
            else:
                time_frame_losses[str(time_frames)].extend([loss_this_time_frame.item()])
                time_frame_labels[str(time_frames)].extend(this_time_frame_ys.detach().cpu().numpy().tolist())
                time_frame_fc_outputs[str(time_frames)].extend(this_time_frame_fc_output.detach().cpu().numpy().tolist())
            time_frame_start = time_frame_start + 5

    avg_loss = total_loss / num_batches
    tpr, fpr, thresholds = sklearn.metrics.roc_curve(y_true = labels, y_score = outputs, pos_label = 1)
    auc = sklearn.metrics.auc(fpr, tpr)
    print(f"Overall Train loss: {avg_loss} , Overall Train AUC: {auc}")

    for time_period in time_frame_losses.keys():
        losses = time_frame_losses[time_period]
        labels = time_frame_labels[time_period]
        outputs_fc = [x[0] for x in time_frame_fc_outputs[time_period]]
        
        avg_loss = sum(losses) / num_batches
        tpr, fpr, thresholds = sklearn.metrics.roc_curve(y_true = labels, y_score = outputs_fc, pos_label = 1)
        auc = sklearn.metrics.auc(fpr, tpr)
        print(f"Train loss for period {time_period}: {avg_loss} , Train AUC for period {time_period}: {auc}")

    return model

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    i = 0
    with torch.no_grad():
        outputs = []
        labels = []
        for X, y in data_loader:
            i += 1
            fc_output = model(X)
            fc_output_long = fc_output.view([data_loader.dataset.sequence_length * X.size(0), model.n_classes])
            y_long = y.view([data_loader.dataset.sequence_length * X.size(0)])
            outputs.extend([x[0] for x in fc_output_long.detach().numpy().tolist()])
            labels.extend(y_long.numpy())
            total_loss += loss_function(fc_output_long, y_long).item()

    time_frame_start = 0
    if data_loader.dataset.sequence_length%5 != 0:
        print('Pick a sequence length divisable by 5')
        return None
    for time_frames in range(int(data_loader.dataset.sequence_length / 5)):
        time_frames += 1
        this_time_frame_fc_output = fc_output[:, time_frame_start : time_frames*5, :].reshape([X.size(0) * 5, 2])
        this_time_frame_ys = y[:, time_frame_start : time_frames*5].reshape([X.size(0) * 5])
        loss_this_time_frame = loss_function(this_time_frame_fc_output, this_time_frame_ys)
        if i == 1:
            time_frame_losses.update({str(time_frames) : [loss_this_time_frame.item()]})
            time_frame_labels.update({str(time_frames) : this_time_frame_ys.detach().cpu().numpy().tolist()})
            time_frame_fc_outputs.update({str(time_frames) : this_time_frame_fc_output.detach().cpu().numpy().tolist()})
        else:
            time_frame_losses[str(time_frames)].extend([loss_this_time_frame.item()])
            time_frame_labels[str(time_frames)].extend(this_time_frame_ys.detach().cpu().numpy().tolist())
            time_frame_fc_outputs[str(time_frames)].extend(this_time_frame_fc_output.detach().cpu().numpy().tolist())
        time_frame_start = time_frame_start + 5

    avg_loss = total_loss / num_batches
    tpr, fpr, thresholds = sklearn.metrics.roc_curve(y_true = labels, y_score = outputs, pos_label = 1)
    auc = sklearn.metrics.auc(fpr, tpr)
    print(f"Overall test loss: {avg_loss} , Overall test AUC: {auc}")

    for time_period in time_frame_losses.keys():
        losses = time_frame_losses[time_period]
        labels = time_frame_labels[time_period]
        outputs_fc = [x[0] for x in time_frame_fc_outputs[time_period]]
        
        avg_loss = sum(losses) / num_batches
        tpr, fpr, thresholds = sklearn.metrics.roc_curve(y_true = labels, y_score = outputs_fc, pos_label = 1)
        auc = sklearn.metrics.auc(fpr, tpr)
        print(f"Test loss for period {time_period}: {avg_loss} , Test AUC for period {time_period}: {auc}")

def train_rush_lstm(train_loader, test_loader, model, loss_function, optimizer, num_epochs = 10):
    for ix_epoch in range(num_epochs):
        print(f"Epoch {ix_epoch}\n---------")
        model = train_model(train_loader, model, loss_function, optimizer=optimizer)
        test_model(test_loader, model, loss_function)
        print()
    return(model)

def predict_play(master_track, play_id, model, normalize = True, replace_player = None):
    week = all_data.search_track_weeks(variables = ["playId"], variable_values = [play_id]).reset_index(drop = True).week.tolist()[0]
    play_dat_label = master_track.get_rush_sequences_labels(week = week + 1, normalize = normalize, play_id = play_id, replace_player = replace_player)
    play_dataset = RushDataset(sequences = play_dat_label, sequence_length=30)
    play_loader = DataLoader(play_dataset, batch_size = 30, shuffle = True)

    outputs = []
    model.eval()
    with torch.no_grad():
        for X, _ in play_loader:
            y_star = model.get_layer_probabilities(X)
            outputs.extend(y_star)
    return outputs








    def plot_metric_averages(self, week):
        if self.overall_avgs.get(str(week - 1)) is None:
            print("Rusher averages are not set to week of this play. Use the load_distance_averages_by_time_after_snap to load proper week averages")
            return(None)
        overall_avgs = self.overall_avgs.get(str(week))
        plt.plot(overall_avgs.time_after_snap.values, overall_avgs.rusher_distance_to_qb_gained_league_avg.values)
        plt.show()





# to do:
# think about the fact that metric must be team based and must incoorperate number of rushers
# fix updater (ie. week 3 or above)
# add qb to tracking plots
# create modeling for each play frame
# add model prob of event to plots
# create funciton to get player metric after week x
# create function to get team metric for week x
# create function to get result for number of team events and training data
# create modeling for team number of events by metric





    














        


    






        











NameError: name 'Dataset' is not defined